# Imports

In [1]:
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date, timedelta
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
from time import sleep
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape, TimeDistributed, LSTM, Lambda, Bidirectional, RepeatVector, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback, EarlyStopping, ProgbarLogger, History
import tensorflow as tf
import keras.backend as K
from IPython.display import display
from tensorflow.keras.metrics import get as metric_get
import re
import talos
from copy import deepcopy, copy
import time
import util
import functools 
import itertools
import networkx as nx
from typing import List, Iterator, Tuple, Dict, Union
import json

# Set up GPU:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

# Load data
load hospitalisations and trends

In [2]:
# topics considered
list_topics = {
    'Fièvre': '/m/0cjf0',
    #'Mal de gorge': '/m/0b76bty',
    #'Dyspnée': '/m/01cdt5',
    #'Agueusie': '/m/05sfr2',
    #'Anosmie': '/m/0m7pl',
    #'Coronavirus': '/m/01cpyy',
    #'Virus': '/m/0g9pc',
    #'Température corporelle humaine': '/g/1213j0cz',
    #'Épidémie': '/m/0hn9s',
    #'Symptôme': '/m/01b_06',
    #'Thermomètre': '/m/07mf1',
    #'Grippe espagnole': '/m/01c751',
    #'Paracétamol': '/m/0lbt3',
    #'Respiration': '/m/02gy9_',
    #'Toux': '/m/01b_21'
}

# hospitalisations features given as input
list_hosp_features = [
    'NEW_HOSP',
    #'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
]

europe = False  # if True, use european countries. Otherwise, use french regions and belgium

Additional information: what is the target, should some features remain unscaled?

In [3]:
# target, should be one of the hosp features
target = 'NEW_HOSP'

# features that should not be scaled
unscaled = [
    #'NEW_HOSP',
    #'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
    #'Fièvre',
    #'Mal de gorge',
    #'Dyspnée',
    #'Agueusie',
    #'Anosmie',
    #'Coronavirus',
    #'Virus',
    #'Température corporelle humaine',
    #'Épidémie',
    #'Symptôme',
    #'Thermomètre',
    #'Grippe espagnole',
    #'Paracétamol',
    #'Respiration',
    #'Toux',
]

Type of prediction: how many days as input sould be used to predict how many days as output? Should we give a prediction on all days or only on the last?

In [4]:
date_begin = "2020-02-01"
n_forecast = 20
n_samples = 30
predict_one = False

In [19]:
validation_metrics = [metric_get("MeanSquaredError"), metric_get('MeanAbsoluteError'), 
                      metric_get('RootMeanSquaredError')]

url_world = "../data/hospi/world.csv"
url_pop = "../data/population.txt"
url_trends = "../data/trends/model/"
url_hospi_belgium = "../data/hospi/be-covid-hospi.csv"
url_department_france = "france_departements.csv"
url_hospi_france_new = "../data/hospi/fr-covid-hospi.csv"
url_hospi_france_tot = "../data/hospi/fr-covid-hospi-total.csv"
if europe:
    population = util.get_world_population(url_pop)
    renaming = {v: k for k, v in util.european_geocodes.items()}
    geocodes = {k: v for k, v in util.european_geocodes.items() if population[k] > 1_000_000}
    df_hospi = util.hospi_world(url_world, geocodes, renaming, new_hosp=True, date_begin=date_begin)
    augment_population = {k: v/1000 for k, v in population.items()}
else:
    population = pd.read_csv(url_department_france).groupby('regionTrends').agg({'population': 'sum'})
    augment_population = {k: pop['population'] / 100_000 for k, pop in population.iterrows()}  # pop per 100.000
    df_hospi = util.hospi_french_region_and_be(url_hospi_france_tot, url_hospi_france_new, url_hospi_belgium, 
                                           url_department_france, util.french_region_and_be, new_hosp_in=True, 
                                           tot_hosp=True, date_begin=date_begin)
for k in df_hospi.keys(): # Rolling average of 7 days 
    df_hospi[k] = df_hospi[k].rolling(7, center=True).mean().dropna()
scaler_generator = StandardScaler
dg = util.DataGenerator(df_hospi, n_samples, n_forecast, 'NEW_HOSP', scaler_generator=scaler_generator, scaler_type='batch',
                       augment_merge=3, augment_adjacency=util.france_region_adjacency, augment_population=augment_population,
                       predict_one=predict_one, cumsum=False)
n_features = dg.n_features
target_idx = dg.target_idx

### Model

In [6]:
if not predict_one:
    weights_loss = np.array([(1/x) for x in range(1, n_forecast+1)])
else:
    weights_loss = 1

def custom_loss_function(y_true, y_pred):
    y_true = y_true * weights_loss
    y_pred = y_pred * weights_loss
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

def get_encoder_decoder(batch_input_shape):
    model = Sequential()
    #model.add(Bidirectional(LSTM(8, return_sequences=True, stateful=False), 
    #                        input_shape=(n_samples, n_features), merge_mode="ave"))
    model.add(LSTM(16, return_sequences=True, stateful=False, batch_input_shape=batch_input_shape, recurrent_dropout=0))
    model.add(LSTM(8, return_sequences=False, stateful=False))
    model.add(RepeatVector(n_forecast))  # repeat
    model.add(LSTM(8, return_sequences=True, stateful=False))  # dec
    if not predict_one:
        model.add(LSTM(16, return_sequences=True, stateful=False))  # dec
        model.add(TimeDistributed(Dense(1)))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(16, return_sequences=False, stateful=False))  # dec
        model.add(Dense(1))
        model.add(Reshape((1,)))
    model.compile(loss=custom_loss_function, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model
    
get_encoder_decoder((1, n_samples, n_features)).output_shape

(1, 20)

## Models to beat
### MultiStepLastBaseline
This model repeats the last value of hospitalisations `n_forecast` time

In [7]:
class MultiStepLastBaseline(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def call(self, inputs):
        a = inputs[:, -1:, target_idx:target_idx+1]  # target of the last days
        # a = tf.where(tf.not_equal(a, 0), tf.zeros_like(a), a)
        if not predict_one:
            return tf.tile(
                a,
                [1, n_forecast, 1]   # repeat target n_forecast time
            )
        else:
            return tf.tile(a, [1, 1, 1])

def get_baseline():
    model = MultiStepLastBaseline()
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### Linear regression
linear regression of the last `n_sample` days used to predict the next `n_forecast` days

In [8]:
class LinearRegressionHospi(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def predict(self, inputs):
        y = inputs[:, :, target_idx]  # target of the last days
        length = len(inputs)
        x = np.arange(n_samples).reshape(-1,1)  # dates of the target
        if not predict_one:
            result = np.zeros((length, n_forecast))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])  # linear regression of (days, target)
                result[i] = regr.predict(np.arange(n_samples, n_samples+n_forecast).reshape(-1,1))
        else:
            result = np.zeros((length, 1))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])
                result[i] = regre.predict([n_samples+n_forecast-1])
        return result
        
def get_custom_linear_regression():
    model = LinearRegressionHospi()
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### 1 fully connected layer (Dense model)
Using only the target in the fit

In [9]:
DenseModel = Sequential()
DenseModel.add(Lambda(lambda x: x[:,:,target_idx]))  # select only the target of the previous days
if not predict_one:
    DenseModel.add(Dense(n_forecast))   # predict the next target based on the previous ones
else:
    DenseModel.add(Dense(1))
DenseModel.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

def get_dense_model(batch_input_shape):
    model = Sequential()
    model.add(Lambda(lambda x: x[:,:,target_idx], batch_input_shape=batch_input_shape))  # select only the target of the previous days
    model.add(Dense(n_forecast))   # predict the next target based on the previous ones
    model.compile(loss=tf.losses.MeanSquaredError(),
                          metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

# Simple prediction
use a percentage of values for training and the remaining values for testing

In [10]:
# X% for training, remaining for test
ratio_training = 0.9
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
test_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_test_unscaled = dg.get_x(test_idx, scaled=False)
Y_test_unscaled = dg.get_y(test_idx, scaled=False)
X_test = dg.get_x(test_idx, scaled=True, use_previous_scaler=True)
Y_test = dg.get_y(test_idx, scaled=True, use_previous_scaler=True)
Y_test_unpadded_unscaled = dg.remove_padded_y(Y_test_unscaled, idx=test_idx)

### Encoder decoder

In [11]:
batch_input = len(train_idx)
encoder_decoder_train = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features))
encoder_decoder_train.summary()
start = time.time()
encoder_decoder_train.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=batch_input)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (324, 30, 16)             1216      
_________________________________________________________________
lstm_5 (LSTM)                (324, 8)                  800       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (324, 20, 8)              0         
_________________________________________________________________
lstm_6 (LSTM)                (324, 20, 8)              544       
_________________________________________________________________
lstm_7 (LSTM)                (324, 20, 16)             1600      
_________________________________________________________________
time_distributed_1 (TimeDist (324, 20, 1)              17        
_________________________________________________________________
reshape_1 (Reshape)          (324, 20)                

In [13]:
batch_input = len(test_idx)
encoder_decoder_prediction = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features) )
encoder_decoder_prediction.set_weights(encoder_decoder_train.get_weights())

Y_predicted = np.squeeze(encoder_decoder_prediction.predict(X_test, batch_size=batch_input))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[0.01182181 0.03292095 0.05239069 0.07486242 0.10219903 0.13145376
 0.15931553 0.18528912 0.21041148 0.23275081 0.25341427 0.27160552
 0.28784532 0.3017836  0.31335848 0.32158675 0.32743756 0.33143487
 0.33252344 0.33246252]
[ 1.8350716   3.14282862  5.36804604  7.84245199 10.50754129 13.16738782
 15.69878695 18.14806111 20.68732077 23.12374419 25.51227921 27.72285965
 29.72751725 31.5347566  33.08889689 34.29457278 35.23735888 35.88072739
 36.19764656 36.38213828]
[ 3.81933408  5.32108304  7.68964482 10.20137742 12.92528528 15.58922774
 18.2787198  20.68907373 23.08679783 25.50786655 27.88145711 30.05661317
 32.12509532 34.01339593 35.6625149  37.14915806 38.34783643 39.41953212
 40.34762178 41.08044124]


### Baseline

In [15]:
baseline_model= get_baseline()
Y_predicted_unscaled = np.squeeze(baseline_model.predict(X_test_unscaled))

print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[  0.46074878  -0.15338165  -0.40157006   0.74939613  -2.39673914
  -2.88345411   0.05736714  -0.38828503  -3.97644928  -6.80736716
  -8.93417875 -12.83595009 -16.29206925 -14.67572464 -17.6491546
 -21.76751208 -27.00664252 -30.76871981 -36.3254831  -36.26026571]
[ 97.23976213 121.77371275 121.59357122 122.7349443  128.44000302
 105.06647095  67.53846734 110.9022132  134.43473352 129.85358327
 136.07685939 141.34590485 126.69369166  95.88279133 132.96303825
 152.38068353 148.63060826 152.68435713 156.96830775 143.07008432]


### Dense model

In [18]:
batch_input_shape = (1, n_samples, n_features)
dense_model = get_dense_model(batch_input_shape)
dense_model.summary()
start = time.time()
dense_model.fit(X_train, Y_train, epochs=400, verbose=1)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (1, 30)                   0         
_________________________________________________________________
dense_4 (Dense)              (1, 20)                   620       
Total params: 620
Trainable params: 620
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
233/233 [==============================] - 0s 1ms/step - loss: 1.5908 - mse: 1.5908 - mae: 0.7549 - root_mean_squared_error: 1.2600
Epoch 2/400
233/233 [==============================] - 0s 1ms/step - loss: 1.0971 - mse: 1.0971 - mae: 0.6273 - root_mean_squared_error: 1.0471
Epoch 3/400
233/233 [==============================] - 0s 1ms/step - loss: 0.9083 - mse: 0.9083 - mae: 0.5514 - root_mean_squared_error: 0.9530
Epoch 4/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8290 - ms

233/233 [==============================] - 0s 1ms/step - loss: 0.7915 - mse: 0.7915 - mae: 0.5078 - root_mean_squared_error: 0.8895
Epoch 110/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8169 - mse: 0.8169 - mae: 0.5109 - root_mean_squared_error: 0.9037
Epoch 111/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8016 - mse: 0.8016 - mae: 0.5088 - root_mean_squared_error: 0.8953
Epoch 112/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5031 - root_mean_squared_error: 0.8907
Epoch 113/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7977 - mse: 0.7977 - mae: 0.5078 - root_mean_squared_error: 0.8930
Epoch 114/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7903 - mse: 0.7903 - mae: 0.5050 - root_mean_squared_error: 0.8889
Epoch 115/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7931 - mse: 0.7931 - mae: 0.5090 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5105 - root_mean_squared_error: 0.8984
Epoch 222/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7810 - mse: 0.7810 - mae: 0.5009 - root_mean_squared_error: 0.8834
Epoch 223/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8222 - mse: 0.8222 - mae: 0.5133 - root_mean_squared_error: 0.9066
Epoch 224/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7952 - mse: 0.7952 - mae: 0.5062 - root_mean_squared_error: 0.8917
Epoch 225/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8022 - mse: 0.8022 - mae: 0.5108 - root_mean_squared_error: 0.8956
Epoch 226/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8027 - mse: 0.8027 - mae: 0.5081 - root_mean_squared_error: 0.8957
Epoch 227/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7823 - mse: 0.7823 - mae: 0.5039 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.7821 - mse: 0.7821 - mae: 0.5055 - root_mean_squared_error: 0.8843
Epoch 334/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7888 - mse: 0.7888 - mae: 0.5043 - root_mean_squared_error: 0.8880
Epoch 335/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8080 - mse: 0.8080 - mae: 0.5086 - root_mean_squared_error: 0.8988
Epoch 336/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5021 - root_mean_squared_error: 0.8905
Epoch 337/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7803 - mse: 0.7803 - mae: 0.4942 - root_mean_squared_error: 0.8828
Epoch 338/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7913 - mse: 0.7913 - mae: 0.5055 - root_mean_squared_error: 0.8895
Epoch 339/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5021 - root_mean_squared_error:

In [ ]:
Y_predicted = np.squeeze(dense_model.predict(X_test, batch_size=1))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(mean_absolute_error(Y_predicted_unpadded_unscaled, Y_test_unpadded_unscaled, multioutput="raw_values"))

In [10]:
def plot_prediction_vs_actual(predicted, idx, loc, horizon=1, show_fit=50, mode=0, map_to_hosp=True):
    """
    plot the prediction done on a specific horizon along with the last points of data before
    :param loc: location of the prediction
    :param predicted: value predicted
    :param idx: index of the y_values used. Must be filtered
    :param horizon: horizon that must be plot
    :param show_fit: max number of days of fit that must be plot
    :param mode: used when horizon > 1. tell how to plot with respect to the horizons
        - mode = 0: plot one graph for all values for this specific horizon
        - mode = 1: plot one graph for all horizon in [1, horizon]
    :param map_to_hosp: if True, show the plot of predictions mapped to the total number of hosp
    """
    target_expected = data_value_region(idx, loc)[1]
    
    if not predict_one:
        if mode == 0:
            horizon_range = range(horizon, horizon+1)
        elif mode == 1:
            horizon_range = range(1, horizon+1)
    else:
        horizon_range = [1]
    
    for horizon in horizon_range:
        fig = plt.figure(figsize=(6,3))
        if show_fit > 0:
            fit_idx = [idx[0]-i for i in range(show_fit, 0, -1) if idx[0]-i >= 0]
            fit_dates = get_y_dates(fit_idx, horizon=horizon)
            # get all predictions of corresponding horizon that were used to fit the model
            fit_values = data_value_region(fit_idx, loc)[1][:,horizon-1].reshape(-1)
            if map_to_hosp:
                fit_values = mapper(fit_values, loc)
            plt.plot(fit_dates, fit_values, label="fitting used")

        prediction_dates = get_y_dates(idx, horizon=horizon)
        if map_to_hosp:
            target_expected_map = mapper(target_expected[:,horizon-1].reshape(-1), loc)
        else:
            target_expected_map = target_expected[:,horizon-1].reshape(-1)
        plt.plot(prediction_dates, target_expected_map, marker=".", label="True value")

        if map_to_hosp:
            prediction = mapper(predicted[:,horizon-1].reshape(-1), loc)
        else:
            prediction = predicted[:,horizon-1].reshape(-1)
        if not predict_one:
            plot_label = f"prediction horizon {horizon}"
        else:
            plot_label = f"prediction horizon {n_forecast}"
        plt.plot(prediction_dates, prediction, marker='.', label=f"prediction horizon {horizon}")

        ax = fig.axes[0]
        # set locator
        if len(idx) + show_fit > 50:  # monthly locator
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        else:  # weekly locator
            ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        # set formatter
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
        # set font and rotation for date tick labels
        plt.gcf().autofmt_xdate()
        plt.legend()
        plt.grid()
        if mode != 0:
            plt.show()
    

In [16]:
class ValidationLogger(ProgbarLogger):
    """
    compute metrics on the validation set, using a different batch size than the training set
    at the end of each epoch, the weights of the training model are used to set the validation model
    the error is computed based on the unscaled and unpadded values
    """
    
    def __init__(self, validation_model, val_batch_size, X_val, Y_val, val_idx, val_geo):
        """
        :param validation_model: validation model ready to be used
        :param X_val: X validation set
        :param Y_val: Y validation set, already unscaled and unpadded
        :param val_idx: validation idx
        :param val_geo: validation geoloc
        """
        super(ValidationLogger, self).__init__(count_mode='steps')
        self.validation_model = validation_model
        self.val_batch_size = val_batch_size
        self.X_val = X_val
        self.Y_val = Y_val
        self.val_idx = val_idx
        self.val_geo = val_geo
    
    def on_epoch_end(self, epoch, logs=None):
        """
        at the end of each epoch, compute the metrics asked on the validation set
        """        
        self.validation_model.set_weights(self.model.get_weights())
        Y_pred = self.validation_model.predict(self.X_val, batch_size=self.val_batch_size)
        Y_pred = dg.inverse_transform_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        Y_pred = dg.remove_padded_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        for metric in validation_metrics:
            metric.reset_states()
            metric.update_state(Y_pred, self.Y_val)
            logs[f"val_{metric.name}"] = metric.result().numpy()  # prepend name for validation set
        super(ValidationLogger, self).on_epoch_end(epoch, logs)
        


In [17]:
def compute_error(Y_expected, Y_actual) -> pd.DataFrame:
    MAE = mean_absolute_error(Y_expected, Y_actual, multioutput="raw_values")
    MSE = mean_squared_error(Y_expected, Y_actual, multioutput="raw_values")
    values = {}
    for t in range(n_forecast):
        values[f'MAE(t+{t+1})'] = [MAE[t]]
        values[f'MSE(t+{t+1})'] = [MSE[t]]
    return pd.DataFrame(values)


def walk_forward_evaluation(model_generator: callable, nb_fit_first: int, nb_validation: int, nb_test: int, 
                            epochs: int = 400, plot=False, verbose: int = 1,
                            return_history: bool = False, batch_size_fun: callable = None,
                            es_stop_val: bool = False
                           ) -> Union[pd.DataFrame, List[History]]:
    """
    evaluates a model using a walk forward evaluation: multiples fit are done, each followed by at most nb_test
    to evaluate the model
    :param model_generator: function returning the model to evaluate
    :param nb_fit_first: number of datapoints used for the first fit
    :param nb_validation: number of datapoints used for each evaluation set
    :param nb_test: number of datapoints used for the test set (at most)
    :param epochs: number of epochs used on each fit
    :param callbacks: list of callbacks to use
    :param verbose: verbose level. Passed to fit and used to display the error dataframe
    :param return_history: if True, returns the list of history of each walk
    :param batch_size_fun: function used to compute the batch size based on the X_train tensor
        if not specified, default to batch_size = len(train_idx)
    """
    # initial index used
    max_len = dg.batch_size
    train_idx = np.arange(nb_fit_first)
    valid_idx = np.arange(nb_fit_first, nb_fit_first + nb_validation)
    if nb_fit_first + nb_validation >= max_len:
        finished = True  # no test can be done
    else:
        finished = False  # a test set can be created
        test_idx = np.arange(nb_fit_first + nb_validation, min(nb_fit_first + nb_validation + nb_test, max_len))
    df_error = pd.DataFrame()
    walk = 0
    last_iter = False  # True when the last iteration is reached
    all_history = []
    while not finished:
        X_train = dg.get_x(train_idx)
        Y_train = dg.get_y(train_idx)
        if batch_size_fun is None:
            batch_size = len(train_idx)
        else:
            batch_size = batch_size_fun(X_train)
            
        if len(valid_idx) > 0:
            X_val = dg.get_x(valid_idx, geo=dg.loc_init)
            Y_val_real = dg.get_y(valid_idx, geo=dg.loc_init, scaled=False)
            Y_val_real = dg.remove_padded_y(Y_val_real, geo=dg.loc_init, idx=valid_idx)
            batch_size_val = len(X_val)
            model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
            val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_real, valid_idx, dg.loc_init)
            if es_stop_val:
                callbacks = [val_log, EarlyStopping(monitor="val_root_mean_squared_error", patience=25)]
            else:
                callbacks = [val_log]
        else:
            callbacks = None
            
        model_train = model_generator(batch_input_shape=(batch_size, n_samples, n_features))

        history = model_train.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=verbose)
        
        # test only on the unaugmented data without padding
        batch_size = len(test_idx)
        # construct a new model and use the previous weights to set it
        model_pred = model_generator(batch_input_shape=(batch_size, n_samples, n_features))
        model_pred.set_weights(model_train.get_weights())
        X_test = dg.get_x(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test = dg.get_y(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test_real = dg.get_y(test_idx, geo=dg.loc_init, scaled=False)
        Y_test_real = dg.remove_padded_y(Y_test_real, geo=dg.loc_init, idx=test_idx)
        Y_predicted = model_pred.predict(X_test, batch_size=batch_size)
        Y_predicted_real = dg.inverse_transform_y(Y_predicted, geo=dg.loc_init, idx=test_idx)
        Y_predicted_real = dg.remove_padded_y(Y_predicted_real, geo=dg.loc_init, idx=test_idx)        
        if return_history:
            # add test metrics to the history for this walk, based on the unpadded unscaled data 
            # except for the loss
            for metric in model_train.metrics:
                metric.reset_states()
                if metric.name == 'loss':
                    metric.update_state(Y_predicted_real, Y_test_real)
                else:
                    # compute metric accross each horizon
                    for i in range(n_forecast):
                        metric.update_state(Y_predicted_real[:, i], Y_test_real[:, i])
                        history.history[f"test_{metric.name}(t+{i+1})"] = [metric.result().numpy()]
                        metric.reset_states()
                    # compute mean of metric on all horizon
                    metric.update_state(Y_predicted_real, Y_test_real)
                # prepend name for test set
                history.history[f"test_{metric.name}"] = [metric.result().numpy()]
            # add number of unpadded datapoints
            history.history['nb_test_datapoints'] = [len(Y_test_real)]
            all_history.append(history)
        if not return_history or verbose != 0:
            # compute the error using the unpadded and unscaled data
            error = compute_error(Y_test_real, Y_predicted_real)
            name = f'walk {walk + 1}'
            error['walk'] = name
            error['nb_test_datapoints'] = len(Y_test_real)
            error['days_train'] = len(train_idx)
            error['days_valid'] = len(valid_idx)
            error['days_test'] = len(test_idx)
            error = error.set_index('walk')
            df_error = df_error.append(error)
        if verbose != 0:
            display(df_error)
        if last_iter:
            finished = True
        # indexes for next fit
        train_idx = np.arange(train_idx[-1] + 1 + nb_test)
        valid_idx += nb_test
        if test_idx[-1] + nb_test >= max_len:  # last iteration, less points can be used for the test set
            last_iter = True  # last iteration to be done
            test_idx = np.arange(test_idx[-1], max_len)
        else:
            test_idx += nb_test
        walk += 1
    return df_error if not return_history else all_history

In [21]:
no_div = lambda x : len(x)
df_errors = walk_forward_evaluation(get_dense_model, 250, 10, 30, epochs=300, verbose=1, batch_size_fun=no_div,
                                   es_stop_val=True)

Epoch 1/300
1/1 [==============================] - 0s 175ms/step - loss: 1.6832 - mse: 1.6832 - mae: 0.9271 - root_mean_squared_error: 1.2974 - val_mean_squared_error: 11212.2246 - val_mean_absolute_error: 53.5582 - val_root_mean_squared_error: 105.8878
Epoch 2/300
1/1 [==============================] - 0s 23ms/step - loss: 1.5803 - mse: 1.5803 - mae: 0.9016 - root_mean_squared_error: 1.2571 - val_mean_squared_error: 10912.8086 - val_mean_absolute_error: 52.7144 - val_root_mean_squared_error: 104.4644
Epoch 3/300
1/1 [==============================] - 0s 28ms/step - loss: 1.5125 - mse: 1.5125 - mae: 0.8842 - root_mean_squared_error: 1.2299 - val_mean_squared_error: 10663.6689 - val_mean_absolute_error: 52.0103 - val_root_mean_squared_error: 103.2650
Epoch 4/300
1/1 [==============================] - 0s 24ms/step - loss: 1.4594 - mse: 1.4594 - mae: 0.8702 - root_mean_squared_error: 1.2081 - val_mean_squared_error: 10448.6641 - val_mean_absolute_error: 51.3982 - val_root_mean_squared_err

1/1 [==============================] - 0s 34ms/step - loss: 0.8760 - mse: 0.8760 - mae: 0.6790 - root_mean_squared_error: 0.9359 - val_mean_squared_error: 7019.0435 - val_mean_absolute_error: 41.0523 - val_root_mean_squared_error: 83.7797
Epoch 34/300
1/1 [==============================] - 0s 23ms/step - loss: 0.8655 - mse: 0.8655 - mae: 0.6747 - root_mean_squared_error: 0.9303 - val_mean_squared_error: 6933.6001 - val_mean_absolute_error: 40.7874 - val_root_mean_squared_error: 83.2682
Epoch 35/300
1/1 [==============================] - 0s 27ms/step - loss: 0.8553 - mse: 0.8553 - mae: 0.6705 - root_mean_squared_error: 0.9248 - val_mean_squared_error: 6849.6538 - val_mean_absolute_error: 40.5240 - val_root_mean_squared_error: 82.7626
Epoch 36/300
1/1 [==============================] - 0s 28ms/step - loss: 0.8454 - mse: 0.8454 - mae: 0.6664 - root_mean_squared_error: 0.9195 - val_mean_squared_error: 6765.3130 - val_mean_absolute_error: 40.2678 - val_root_mean_squared_error: 82.2515
Epoch

Epoch 66/300
1/1 [==============================] - 0s 23ms/step - loss: 0.6444 - mse: 0.6444 - mae: 0.5709 - root_mean_squared_error: 0.8027 - val_mean_squared_error: 4771.1758 - val_mean_absolute_error: 34.6145 - val_root_mean_squared_error: 69.0737
Epoch 67/300
1/1 [==============================] - 0s 30ms/step - loss: 0.6399 - mse: 0.6399 - mae: 0.5685 - root_mean_squared_error: 0.7999 - val_mean_squared_error: 4724.2842 - val_mean_absolute_error: 34.4843 - val_root_mean_squared_error: 68.7334
Epoch 68/300
1/1 [==============================] - 0s 24ms/step - loss: 0.6355 - mse: 0.6355 - mae: 0.5661 - root_mean_squared_error: 0.7972 - val_mean_squared_error: 4679.1099 - val_mean_absolute_error: 34.3603 - val_root_mean_squared_error: 68.4040
Epoch 69/300
1/1 [==============================] - 0s 22ms/step - loss: 0.6312 - mse: 0.6312 - mae: 0.5637 - root_mean_squared_error: 0.7945 - val_mean_squared_error: 4635.1689 - val_mean_absolute_error: 34.2401 - val_root_mean_squared_error: 

1/1 [==============================] - 0s 27ms/step - loss: 0.5350 - mse: 0.5350 - mae: 0.5092 - root_mean_squared_error: 0.7315 - val_mean_squared_error: 3850.1060 - val_mean_absolute_error: 32.2608 - val_root_mean_squared_error: 62.0492
Epoch 99/300
1/1 [==============================] - 0s 23ms/step - loss: 0.5324 - mse: 0.5324 - mae: 0.5075 - root_mean_squared_error: 0.7297 - val_mean_squared_error: 3840.2109 - val_mean_absolute_error: 32.2542 - val_root_mean_squared_error: 61.9694
Epoch 100/300
1/1 [==============================] - 0s 23ms/step - loss: 0.5299 - mse: 0.5299 - mae: 0.5061 - root_mean_squared_error: 0.7279 - val_mean_squared_error: 3825.5027 - val_mean_absolute_error: 32.2253 - val_root_mean_squared_error: 61.8507
Epoch 101/300
1/1 [==============================] - 0s 26ms/step - loss: 0.5274 - mse: 0.5274 - mae: 0.5043 - root_mean_squared_error: 0.7262 - val_mean_squared_error: 3820.0044 - val_mean_absolute_error: 32.2411 - val_root_mean_squared_error: 61.8062
Epo

1/1 [==============================] - 0s 23ms/step - loss: 0.4686 - mse: 0.4686 - mae: 0.4656 - root_mean_squared_error: 0.6846 - val_mean_squared_error: 3854.5623 - val_mean_absolute_error: 33.4133 - val_root_mean_squared_error: 62.0851
Epoch 131/300
1/1 [==============================] - 0s 27ms/step - loss: 0.4670 - mse: 0.4670 - mae: 0.4645 - root_mean_squared_error: 0.6834 - val_mean_squared_error: 3869.8701 - val_mean_absolute_error: 33.5014 - val_root_mean_squared_error: 62.2083
Epoch 132/300
1/1 [==============================] - 0s 27ms/step - loss: 0.4655 - mse: 0.4655 - mae: 0.4635 - root_mean_squared_error: 0.6822 - val_mean_squared_error: 3876.5222 - val_mean_absolute_error: 33.5526 - val_root_mean_squared_error: 62.2617
Epoch 133/300
1/1 [==============================] - 0s 22ms/step - loss: 0.4639 - mse: 0.4639 - mae: 0.4622 - root_mean_squared_error: 0.6811 - val_mean_squared_error: 3895.4526 - val_mean_absolute_error: 33.6507 - val_root_mean_squared_error: 62.4136
Ep

...

Epoch 34/300
1/1 [==============================] - 0s 22ms/step - loss: 1.0753 - mse: 1.0753 - mae: 0.7665 - root_mean_squared_error: 1.0370 - val_mean_squared_error: 5846.6675 - val_mean_absolute_error: 39.0822 - val_root_mean_squared_error: 76.4635
Epoch 35/300
1/1 [==============================] - 0s 22ms/step - loss: 1.0609 - mse: 1.0609 - mae: 0.7609 - root_mean_squared_error: 1.0300 - val_mean_squared_error: 5785.9502 - val_mean_absolute_error: 38.9443 - val_root_mean_squared_error: 76.0654
Epoch 36/300
1/1 [==============================] - 0s 25ms/step - loss: 1.0468 - mse: 1.0468 - mae: 0.7554 - root_mean_squared_error: 1.0232 - val_mean_squared_error: 5727.5566 - val_mean_absolute_error: 38.8111 - val_root_mean_squared_error: 75.6806
Epoch 37/300
1/1 [==============================] - 0s 22ms/step - loss: 1.0331 - mse: 1.0331 - mae: 0.7499 - root_mean_squared_error: 1.0164 - val_mean_squared_error: 5670.0444 - val_mean_absolute_error: 38.6811 - val_root_mean_squared_error: 

1/1 [==============================] - 0s 22ms/step - loss: 0.7526 - mse: 0.7526 - mae: 0.6310 - root_mean_squared_error: 0.8675 - val_mean_squared_error: 4566.2822 - val_mean_absolute_error: 36.8426 - val_root_mean_squared_error: 67.5743
Epoch 67/300
1/1 [==============================] - 0s 22ms/step - loss: 0.7461 - mse: 0.7461 - mae: 0.6279 - root_mean_squared_error: 0.8638 - val_mean_squared_error: 4542.9966 - val_mean_absolute_error: 36.8291 - val_root_mean_squared_error: 67.4018
Epoch 68/300
1/1 [==============================] - 0s 22ms/step - loss: 0.7397 - mse: 0.7397 - mae: 0.6248 - root_mean_squared_error: 0.8600 - val_mean_squared_error: 4521.2490 - val_mean_absolute_error: 36.8193 - val_root_mean_squared_error: 67.2402
Epoch 69/300
1/1 [==============================] - 0s 22ms/step - loss: 0.7334 - mse: 0.7334 - mae: 0.6217 - root_mean_squared_error: 0.8564 - val_mean_squared_error: 4499.9771 - val_mean_absolute_error: 36.8110 - val_root_mean_squared_error: 67.0819
Epoch

Epoch 99/300
1/1 [==============================] - 0s 23ms/step - loss: 0.6005 - mse: 0.6005 - mae: 0.5506 - root_mean_squared_error: 0.7749 - val_mean_squared_error: 4169.3218 - val_mean_absolute_error: 37.0094 - val_root_mean_squared_error: 64.5703
Epoch 100/300
1/1 [==============================] - 0s 28ms/step - loss: 0.5974 - mse: 0.5974 - mae: 0.5488 - root_mean_squared_error: 0.7729 - val_mean_squared_error: 4166.3701 - val_mean_absolute_error: 37.0340 - val_root_mean_squared_error: 64.5474
Epoch 101/300
1/1 [==============================] - 0s 23ms/step - loss: 0.5943 - mse: 0.5943 - mae: 0.5471 - root_mean_squared_error: 0.7709 - val_mean_squared_error: 4162.0762 - val_mean_absolute_error: 37.0623 - val_root_mean_squared_error: 64.5142
Epoch 102/300
1/1 [==============================] - 0s 26ms/step - loss: 0.5914 - mse: 0.5914 - mae: 0.5454 - root_mean_squared_error: 0.7690 - val_mean_squared_error: 4159.7534 - val_mean_absolute_error: 37.0908 - val_root_mean_squared_erro

1/1 [==============================] - 0s 25ms/step - loss: 0.5235 - mse: 0.5235 - mae: 0.5058 - root_mean_squared_error: 0.7236 - val_mean_squared_error: 4205.5664 - val_mean_absolute_error: 38.5137 - val_root_mean_squared_error: 64.8503
Epoch 132/300
1/1 [==============================] - 0s 23ms/step - loss: 0.5217 - mse: 0.5217 - mae: 0.5040 - root_mean_squared_error: 0.7223 - val_mean_squared_error: 4216.6099 - val_mean_absolute_error: 38.6125 - val_root_mean_squared_error: 64.9354
Epoch 133/300
1/1 [==============================] - 0s 23ms/step - loss: 0.5199 - mse: 0.5199 - mae: 0.5036 - root_mean_squared_error: 0.7211 - val_mean_squared_error: 4217.2114 - val_mean_absolute_error: 38.6462 - val_root_mean_squared_error: 64.9401
Epoch 134/300
1/1 [==============================] - 0s 22ms/step - loss: 0.5182 - mse: 0.5182 - mae: 0.5018 - root_mean_squared_error: 0.7198 - val_mean_squared_error: 4228.2617 - val_mean_absolute_error: 38.7405 - val_root_mean_squared_error: 65.0251
Ep

,MAE(t+1),MSE(t+1),MAE(t+2),MSE(t+2),MAE(t+3),MSE(t+3),MAE(t+4),MSE(t+4),MAE(t+5),MSE(t+5),...,MAE(t+18),MSE(t+18),MAE(t+19),MSE(t+19),MAE(t+20),MSE(t+20),nb_test_datapoints,days_train,days_valid,days_test
walk,,,,,,,,,,,,,,,,,,,,,
walk 1,90.929552,28418.503067,83.032830,23204.770026,83.339676,23139.828415,83.801911,23291.761082,81.121293,22144.760842,...,50.005786,7075.855486,47.040145,5942.501459,46.116520,5654.791382,690,250,10,30
walk 2,57.431387,7129.226980,49.346735,5032.498466,53.913558,6220.399446,52.120782,5609.448180,46.756107,4413.316944,...,38.985485,2380.483403,38.545709,2371.251378,36.731191,2110.573121,690,280,10,30
walk 3,73.224730,38634.885481,76.708825,40018.193349,76.818426,42392.647383,78.974664,43327.709714,53.825533,20036.219653,...,40.993932,11615.159584,19.663141,1129.058371,40.084055,10386.007430,690,310,10,30
walk 4,70.465531,9841.706838,68.454025,8152.128445,64.591759,7586.935941,77.250694,12813.094885,46.814064,3820.931931,...,39.308206,3094.973028,48.236789,6124.530839,44.361185,5285.396000,253,340,10,11


In [53]:
tf.keras.metrics.get('rmse')

ValueError: Unknown metric function: rmse

In [48]:
model.metrics[3]

In [83]:
# X% for training, remaining for test
ratio_training = 0.8
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
val_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_val = dg.get_x(val_idx, scaled=True, use_previous_scaler=True)
Y_val_unscaled = dg.get_y(val_idx, geo=dg.loc_init, scaled=False, use_previous_scaler=True)
Y_val_unpadded_unscaled = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)

model_generator = get_dense_model

batch_size_train = len(train_idx)
batch_size_val = len(X_val)
model = model_generator(batch_input_shape=(batch_size_train, n_samples, n_features))
model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_unpadded_unscaled, val_idx, dg.loc_init)
callbacks = [val_log, EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=1)]
    
start = time.time()
history = model.fit(X_train, Y_train, batch_size=batch_size_train, epochs=200, callbacks=callbacks)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Epoch 1/200
179/179 [==============================] - 0s 1ms/step - loss: 1.1839 - mse: 1.1839 - mae: 0.7759 - root_mean_squared_error: 1.0793 - val_mean_squared_error: 331.9821 - val_mean_absolute_error: 11.6835 - val_root_mean_squared_error: 18.2204
Epoch 2/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4572 - mse: 0.4572 - mae: 0.4603 - root_mean_squared_error: 0.6760 - val_mean_squared_error: 294.8744 - val_mean_absolute_error: 11.0358 - val_root_mean_squared_error: 17.1719
Epoch 3/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3894 - mse: 0.3894 - mae: 0.4078 - root_mean_squared_error: 0.6240 - val_mean_squared_error: 282.7642 - val_mean_absolute_error: 10.7674 - val_root_mean_squared_error: 16.8156
Epoch 4/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3648 - mse: 0.3648 - mae: 0.3878 - root_mean_squared_error: 0.6040 - val_mean_squared_error: 296.3748 - val_mean_absolute_error: 11.0865 - val_root_mean_squared_err

In [64]:
model.val_metrics[0]([1], [1]).numpy()

0

In [65]:
tf.keras.metrics.get('MeanSquaredError')

In [92]:
hyper_parameter = {
    'hidden_1': [4, 8, 16],
    'hidden_2': [0, 4, 8, 16],
    'lr': [0.001, 0.01],
    'activation': ['relu', 'elu'],
    'reg': [lambda x: regularizers.l1(l=x), lambda x: None],
    'regw': [5e-4, 1e-3],
    'optimizer': ['Adam', 'RMSprop'],
    'losses': ['mse', 'mae', custom_loss_function],
    'scaling': [MinMaxScaler, StandardScaler],
    'batch_size_div': [1],
    'epochs': [600]
}

def get_encoder_decoder(batch_input_shape, p):
    model = Sequential()
    model.add(LSTM(p['hidden_1'], return_sequences=(p['hidden_2'] != 0), 
                   batch_input_shape=batch_input_shape, kernel_regularizer=p['reg'](p['regw'])))
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))
    model.add(RepeatVector(n_forecast))  # repeat
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
    if not predict_one:
        model.add(LSTM(p['hidden_1'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(TimeDistributed(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation'])))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(p['hidden_1'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation']))
        model.add(Reshape((1,)))
    model.compile(loss=p["losses"], optimizer=p["optimizer"], metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    K.set_value(model.optimizer.learning_rate, p['lr'])
    return model
    

def talos_walk(x_train, y_train, x_val, y_val, p):
    """
    walk forward evaluation used by talos to determine the best model to keep
    """
    dg.set_scaler(p['scaling'])
    model_generator = lambda batch_input_shape: get_encoder_decoder(batch_input_shape, p)

    batch_size_fun = lambda x : int(x.shape[0] / p['batch_size_div'])
    history_list = walk_forward_evaluation(model_generator, nb_fit_first=250, nb_validation=0, nb_test=30, 
                                      epochs=p['epochs'], plot=False, verbose=0, 
                                      batch_size_fun=batch_size_fun, return_history=True, es_stop_val=True)
    # compute history to return, based on the list of history
    history = History()
    # used to trick talos to compute the right amount of round_epochs. needs to be the first entry of the dict
    # create an array of len == mean of number of epoch of each history
    history.history['ep'] = np.arange(int(np.mean([len(hist.history['loss']) for hist in history_list]))) + 1
    for log in history_list[0].history.keys():
        history.history[log] = [np.mean([hist.history[log][-1] for hist in history_list])]
    return history, model_generator((1, n_samples, n_features))

scan_object = talos.Scan(
    x=[],
    y=[],
    x_val=[],
    y_val=[],
    params=hyper_parameter,
    model=talos_walk,
    experiment_name='trends1', 
    fraction_limit=0.01
)

100%|██████████| 23/23 [1:07:28<00:00, 176.04s/it]


In [95]:
analyze_object = talos.Analyze(scan_object)
print("MAE", analyze_object.low('mae'))
print("MSE", analyze_object.low('mse'))
print("RMSE", analyze_object.low('root_mean_squared_error'))
print("TEST MAE", analyze_object.low('test_mae'))
print("TEST MSE", analyze_object.low('test_mse'))
print("TEST RMSE", analyze_object.low('test_root_mean_squared_error'))
df = analyze_object.table('test_mae', ascending=True)
print(df.columns)
columns = [f"test_mae(t+{i})" for i in range(1, n_forecast+1)]
df[['hidden_1', 'hidden_2', 'lr', 'scaling', 'test_mse', 'test_mae'] + columns]

MAE 0.043379394337534904
MSE 0.006718363147228956
RMSE 0.0817974004894495
TEST MAE 12.322504043579102
TEST MSE 424.1562194824219
TEST RMSE 19.598430633544922
Index(['lr', 'test_mae(t+4)', 'test_mae(t+9)', 'test_mse(t+13)',
       'test_mse(t+17)', 'test_mae(t+3)', 'test_mse(t+3)', 'test_mse(t+12)',
       'hidden_2', 'test_mae(t+18)', 'test_root_mean_squared_error(t+7)',
       'test_root_mean_squared_error(t+6)',
       'test_root_mean_squared_error(t+15)',
       'test_root_mean_squared_error(t+17)', 'test_mae(t+16)', 'losses',
       'test_root_mean_squared_error(t+14)', 'test_mae(t+2)', 'test_mse(t+4)',
       'test_mae(t+1)', 'test_mse(t+8)', 'test_mse(t+19)', 'mse', 'duration',
       'batch_size_div', 'test_mse(t+14)', 'test_root_mean_squared_error(t+3)',
       'test_root_mean_squared_error(t+11)',
       'test_root_mean_squared_error(t+19)', 'nb_test_datapoints',
       'test_root_mean_squared_error', 'test_mse(t+2)', 'test_mae(t+12)',
       'test_mse(t+11)', 'test_root_mean_

,hidden_1,hidden_2,lr,scaling,test_mse,test_mae,test_mae(t+1),test_mae(t+2),test_mae(t+3),test_mae(t+4),...,test_mae(t+11),test_mae(t+12),test_mae(t+13),test_mae(t+14),test_mae(t+15),test_mae(t+16),test_mae(t+17),test_mae(t+18),test_mae(t+19),test_mae(t+20)
4,8,0,0.001,<class 'sklearn.preprocessing._data.StandardSc...,424.156219,12.322504,5.113103,6.668501,8.047836,9.198174,...,13.257397,13.544383,13.805571,14.080258,14.412364,14.773132,15.134119,15.491337,15.813452,16.175266
3,8,0,0.010,<class 'sklearn.preprocessing._data.MinMaxScal...,461.206116,12.438999,2.725237,4.229722,5.651115,6.975208,...,13.687336,14.325952,14.951972,15.529829,16.136250,16.737545,17.320147,17.887159,18.461542,18.997755
0,16,0,0.010,<class 'sklearn.preprocessing._data.StandardSc...,697.320862,13.102798,5.190972,6.328577,7.232556,8.037759,...,13.692082,14.461799,15.183695,15.908325,16.639370,17.394108,18.143715,18.844168,19.475594,20.034653
22,16,0,0.010,<class 'sklearn.preprocessing._data.StandardSc...,594.379639,13.517742,5.322759,5.749920,7.206419,8.470683,...,15.279795,15.884229,16.384600,16.734604,16.997005,17.237675,17.479227,17.692469,17.879229,18.055279
16,16,8,0.010,<class 'sklearn.preprocessing._data.MinMaxScal...,522.346436,13.588950,6.359708,6.524685,7.138686,7.934609,...,13.750313,14.711052,15.681072,16.623653,17.536077,18.451939,19.357836,20.228899,21.122440,22.027515
12,8,0,0.001,<class 'sklearn.preprocessing._data.MinMaxScal...,575.319824,13.649181,4.518685,6.380329,7.709112,8.785776,...,14.514085,15.175107,15.826385,16.451441,17.089672,17.741415,18.364426,18.960075,19.519262,20.065390
6,8,8,0.001,<class 'sklearn.preprocessing._data.MinMaxScal...,500.516144,13.904951,10.407436,6.431574,7.028675,8.126210,...,14.611106,15.336163,16.041670,16.710649,17.364716,18.019814,18.643921,19.219770,19.770044,20.303814
8,16,0,0.010,<class 'sklearn.preprocessing._data.StandardSc...,852.186157,14.367056,4.004116,4.888426,5.717895,6.713186,...,14.475338,15.735716,17.073145,18.416533,19.727028,21.015491,22.268200,23.456362,24.598537,25.668140
1,4,0,0.001,<class 'sklearn.preprocessing._data.MinMaxScal...,525.906616,14.393981,22.603771,8.642357,6.704407,7.813111,...,15.227543,15.908760,16.431070,16.832075,17.188753,17.494455,17.750347,17.995251,18.228586,18.444473
5,16,8,0.010,<class 'sklearn.preprocessing._data.MinMaxScal...,596.573975,14.586052,8.144485,8.452685,8.734910,9.260450,...,14.801952,15.680233,16.563116,17.415600,18.271658,19.082069,19.828577,20.509150,21.133707,21.685873
